# Analyses performed in the Manuscript

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import utils as utl
import matplotlib.pyplot as plt

from matplotlib import cm
from matplotlib import colors
import matplotlib.ticker as mticker
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline
from scipy.ndimage import gaussian_filter1d   
from functools import partial

import gc

import warnings
import re
import importlib

import data_processing as dp
from data_processing import didx

import common_plot_parameters as cpprm

idx = pd.IndexSlice

# Dark-Light transition
## Main figure

In [ ]:
# #### DATA IMPORT ####

# Import raw data
EXPERIMENT_PATH="data/20250227_DarkLightTransition_25umol"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 300,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

# Set the ylim of the rightmost plots to zero
for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# DCMU
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
EXPERIMENT_PATH="data/20250224_DCMU"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    # colornorm_function=colors.LogNorm,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Set the grid and x-axis formatter for the rightmost plots log axis
for ax in axes[:,-1]:
    ax.grid(visible=True, which="both", axis="y")
    ax.grid(visible=True, which="both", axis="x")
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(utl.log_tick_formatter))

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 200,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    include_analysis=False,
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

for ax in axes.flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# Glycolaldehyde (GA)
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
EXPERIMENT_PATH="data/20250301_OJIP_GA"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
# Get the base plot "AL {treatment:d}s"
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Add arrow marking the timing of GA application
for ax in axes[:,-1]:
    utl.add_application_arrow(ax, x=2, offset=0.02, arrow_frac_len=0.13)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 200,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
# Get the base plot "AL {treatment:d}s"
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

# Set the y-axis of the rightmost plots to start at 0
for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# High Light
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
EXPERIMENT_PATH="data/20250301_OJIP_High_light"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 300,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

# Set the y-axis of the rightmost plots to start at 0
for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# Potassium-Cyanide (KCN)
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
EXPERIMENT_PATH="data/20250312_OJIP_KCN"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Add arrow marking the timing of KCN application
for ax in axes[:,-1]:
    utl.add_application_arrow(ax, x=2, offset=0.03, arrow_frac_len=0.13)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 300,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

# Set the y-axis of the rightmost plots to start at 0
for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# Methylviologen (MV) - 1mM (high concentration)
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
EXPERIMENT_PATH="data/20250314_OJIP_MV_1mM"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not (x.name.startswith("_") or re.search("250uM", x.name))]

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 200,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

# Set the y-axis of the rightmost plots to start at 0
for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# Methylviologen (MV) - 250µM (low concentration)
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
FILEPATHS = []

EXPERIMENT_PATH="data/20250314_OJIP_MV_1mM"
FILEPATHS.extend([x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not (x.name.startswith("_") or re.search("1mM", x.name))])

EXPERIMENT_PATH="data/20250227_MV"
FILEPATHS.extend([x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not (x.name.startswith("_") or re.search("Chlo", x.name))])

ojip, levels = dp.load_data(FILEPATHS)

# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters
)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 200,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters
)

# Set the y-axis of the rightmost plots to start at 0
for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")

# Dark adapted +- Methylviologen (MV) (1mM)
## Main figure

In [ ]:
#### DATA IMPORT ####

# Import raw data
# Get data from 1mM MV experiment and not 250uM MV experiment
EXPERIMENT_PATH="data/20250227_DarkLightTransition_25umol"
FILEPATHS_NOMV = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

EXPERIMENT_PATH="data/20250314_OJIP_MV_1mM"
FILEPATHS_MV = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not (x.name.startswith("_") or re.search("250uM", x.name))]

FILEPATHS = FILEPATHS_NOMV + FILEPATHS_MV

_ojip, levels = dp.load_data(FILEPATHS)

# Replace the treatement with 0 and 1 as the MV treatment concentrations
_ojip_mv = _ojip.loc[:,didx("MV1mM", treatment=[0])]
_ojip_mv.columns = _ojip_mv.columns.set_levels([1], level=3)

# Set the correct treatment levels in the level list
levels["treatments"] = pd.Series({key:[0,1] for key, val in levels["treatments"].to_dict().items()})

# Select the right data and concatenate
ojip = pd.concat([
    _ojip.loc[:,didx("DarkLight", treatment=[0])],
    _ojip_mv
], axis=1)

del _ojip
gc.collect()

# Set the plot parameters
plot_parameters = {
    'treatment_format': 'MV treated',
    'treatment_0_label': 'Untreated',
    'treatment_label': 'CO$_2$ concentration',
    'treatments_select': [0, 1],
    'treatment_center': 0.5,
    'strain_map': {
        'Syn': '$\\mathit{\\boldsymbol{Synechocystis}}$ dark-acclimated + 1mM MV',
        'Chlo': '$\\mathit{\\boldsymbol{Chlorella}}$ dark-acclimated + 1mM MV'
    }
  }

#### PARAMETER CALCULATION ####

# Get the main points of the OJIP curves
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

# Double normalize the data
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

# Calculate the mean and sd between the replicates of the normalized OJIP curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

# Get VJ at common times
VJ_timing, VJ_timing_range, VJ_values = dp.get_common_time_VJ(
    ojip_points=ojip_points,
    ojip_norm=ojip_norm,
    levels=levels
)
VJ_values.to_csv(f"results/VJvalues_{EXPERIMENT_PATH.split("/")[1]}_comparison.csv")

# Calculate the mean and sd between the replicates of the VJ values
VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the experimental light phases for annotation of the plots
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot
fig, axes = utl.get_base_plot_MV(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_comparison.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
#### PARAMETER CALCULATION ####

# Use the default options for finding the FP timing but except for the minimum FP timing
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 200,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

# Extract the FP values and timing
FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

# Calculate the mean and sd between the replicates of the raw OJIP curves
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

#### PLOTTING ####

# Get the base plot
fig, axes = utl.get_base_plot_MV(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.cool,
    light_phases=light_phases,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)


for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# Save the plot
for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_comparison_SI.{ext}", bbox_inches="tight")

# Measurement of VJ depending on the Device, Culture Density, and Light intensity
## Plot of double normalized OJIP

In [ ]:
#### DATA IMPORT ####

# Create overall container for ojip data
ojip = ind = None

# import raw data
FILEPATHS = [x for x in Path("data/20250223_SPIntensityAndDensityComparision").glob("*") if x.is_dir() and not x.name.startswith("_")]

_ojips = _inds= [None] * len(FILEPATHS)

for j, (FILEPATH, _ojip, _ind) in enumerate(zip(FILEPATHS, _ojips, _inds)):
    files = list(Path(FILEPATH).glob("*_OJIP_*.*"))
    print(f"Importing: {FILEPATH}\t\t|  {len(files)} files")

    # Read in the sample ODs
    ods = pd.read_excel(Path(FILEPATH.parent) / "ODs, chl_a.xlsx", index_col=[0,1,2])

    # # Read in the remeasured samples
    remeasured_samples = pd.read_excel(Path(FILEPATH.parent) / "remeasurements.xlsx", index_col=0)

    # Read the OJIP files
    for i, file in enumerate(files):     
        # FL6000 measurement
        if FILEPATH.name.startswith("FL6000"):
            # Read file
            _df = pd.read_table(file, sep="\t", skiprows=17, header=None)
            _df = _df.set_index(0)

        # AquaPen measurement
        elif FILEPATH.name.startswith("AquaPen"):
            # Read file
            _df = pd.read_table(file, sep="\t", index_col=0, skiprows=8, header=None, skipfooter = 38, engine="python").iloc[:,:-1]

        elif FILEPATH.name.startswith("MCPAM"):
            try:
                _df = pd.read_csv(file, sep=";", index_col=0)[["Fluo, V"]]
            except UnicodeDecodeError as e:
                print(f"File corrupted: {file}")
            except Exception as e:
                raise RuntimeError(e)
            

        # Create a common, formatted dataframe
        try:
            _ojip, _ind = utl.add_and_format_df(
                df=_df,
                index=i,
                full_df=_ojip,
                ind=_ind,
                files=files,
                index_fields=["Strain", "Measurement", "Dilution", "Light_intensity", "Device"],
                remeasured_df=remeasured_samples
            )
        except Exception as e:
            print(f"{file.name}\n{e}")
    # Drop missing columns
    _ojip = _ojip.dropna(axis=1)
    _ind = _ind.dropna(axis=1)

    # Add the ODs as index
    _ind.loc["Dilution"] = _ind.loc["Dilution"].str.removeprefix("Dil").astype(int)

    _device = _ind.iloc[-1,0]
    _strain = _ind.iloc[0,0]
    _ind.loc["OD680"] = ods.loc[idx[_strain, _device, :], "OD680"].droplevel([0,1]).loc[_ind.loc["Dilution"].astype(int).to_numpy()].to_numpy()

    # Remove the prefixes from the columns
    _ind.loc["Light_intensity"] = _ind.loc["Light_intensity"].str.removeprefix("Int").astype(int)

    # Set the correct elements and order of the index
    _ind = _ind.loc[["Device", "Strain", "Dilution", "OD680", "Light_intensity"]]

    # Set the index
    _ojip.columns = pd.MultiIndex.from_frame(_ind.T)
    _ojip = _ojip.sort_index()

    if FILEPATH.name.startswith("FL6000"):
        _ojip.index = _ojip.index * 1e3
    elif FILEPATH.name.startswith("AquaPen"):
        _ojip.index = _ojip.index * 1e-3

    _ojip = _ojip.sort_index(axis=1)

    # Exclude early timepoints
    _ojip = _ojip.loc[1e-20:]

    _ojips[j]=_ojip

# Collect all ojips
ojip = pd.concat(_ojips, axis=1)
ojip = ojip.sort_index()

# Exclude saturated AquaPen samples
saturated_aquapen = (ojip[["AquaPen"]].max() == ojip[["AquaPen"]].max().max())
ojip = ojip.drop(saturated_aquapen.index[saturated_aquapen], axis=1)

# Get the strains, ods and light intensities
strains = list(ojip.columns.levels[1])
light_intensities = pd.DataFrame(
    {
        "syn":ojip.columns.levels[-2].to_numpy() / 100 * 6500,
        "chlo":ojip.columns.levels[-2].to_numpy() / 100 * 7500
    },
    index = ojip.columns.levels[-2]
).astype(int)

devices = ojip.columns.levels[0].to_numpy()
dilutions = ojip.columns.levels[3].to_numpy()
dilution_steps = ods.index.levels[-1].to_numpy()

# Define the devices max light intensities
max_settings = pd.DataFrame({ # red / blue actinic
    "FL6000": np.array([100, 100]), 
    "MCPAM":  np.array([19,20]),
    "AquaPen": np.array([93,75]),
}, index=["syn", "chlo"])

max_intensities = pd.DataFrame({ # red / blue actinic
    "FL6000": np.array([6500, 7500]), 
    "MCPAM":  np.array([3256,3713]),
    "AquaPen": np.array([2600,3750]),
}, index=["syn", "chlo"])

intensity_conversion = max_intensities / max_settings

# Get the unique light intensity setting for each device and the respective light intensities
light_intensities = {}

for device in devices:
    for strain in strains:
        _light_intensities = ojip.columns.to_frame().loc[idx[device, strain,:,:],"Light_intensity"].unique()
        light_intensities[(device, strain)] = pd.Series(_light_intensities * intensity_conversion.loc[strain, device], index=_light_intensities)
light_intensities = pd.DataFrame(light_intensities)

# Remove the first datapoint of AquaPen
ojip.loc[ojip.loc[:,"AquaPen"].dropna().index[0],"AquaPen"] = np.nan

#### CALCULATE PARAMETERS ####
# Get the main points of the OJIP curve

ojip_features = pd.DataFrame(index=ojip.columns, columns=["F0", "FM", "Fv", "Fv/FM", "FJapprox", "VJapprox"])
ojip_features = ojip_features.sort_index()

for device in devices:
    _ind = ojip_features.loc[idx[[device]]].index

    # Determine F0
    ojip_features.loc[_ind, "F0"] = ojip.loc[:0.05, _ind].dropna().mean()

    # Determine FM
    ojip_features.loc[_ind, "FM"] = ojip.loc[:, _ind].dropna().rolling(5).mean().max()

    # Determine Fv
    ojip_features.loc[_ind, "Fv"] = ojip_features["FM"] - ojip_features["F0"]

    # Calculate Fv/Fm
    ojip_features.loc[_ind, "Fv/FM"] = ojip_features["Fv"] / ojip_features["FM"]


    # Determine FJ approximately as the maximum
    ojip_features.loc[_ind, "FJapprox"] = ojip.loc[0.1:5, _ind].dropna().rolling(5).mean().max()

    # Determine FJ approximately as the maximum
    ojip_features.loc[_ind, "VJapprox"] = (ojip_features["FJapprox"] - ojip_features["F0"]) / ojip_features["Fv"]

# Double normalize the data
ojip_norm = ojip - ojip_features["F0"]
ojip_norm = ojip_norm / (ojip_features["FM"] - ojip_features["F0"])

ojip_norm = ojip_norm.astype(float)

# Define the time ranges for finding OJIP points
feature_finding_options =     {
        "FJ_time_min" : 0.5, # 0.1  # (float(str(request.form.get('FJ_time_min')))) 
        "FJ_time_max" : 30, # 30   # (float(str(request.form.get('FJ_time_max')))) 
        "FI_time_min" : 10,   # (float(str(request.form.get('FI_time_min')))) 
        "FI_time_max" : 500,  # (float(str(request.form.get('FI_time_max')))) 
        "FP_time_min" : 100,  # (float(str(request.form.get('FP_time_min')))) 
        "FP_time_max" : 1000, # (float(str(request.form.get('FP_time_max')))) 
    }

# Get the ojip pints
ojip_points_res = {}

with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore")
    for device in devices:
        ojip_points_res[device] = utl.determine_OJIP_points(
        ojip_norm[device].dropna(),
        **feature_finding_options,
        )
        
        ojip_points_res[device]["points"].loc[:,("inflection", "FJ_value")].to_csv(f"results/VJvalues_spintensitydensity2_{device}.csv")


#### PLOTTING ####
# Double normalized OJIP plots

# Values for shading
FJ_time_min = feature_finding_options["FJ_time_min"]  # (float(str(request.form.get('FJ_time_min')))) 
FJ_time_max = feature_finding_options["FJ_time_max"]   # (float(str(request.form.get('FJ_time_max')))) 

ojip_mean = ojip_norm

ojip_max = ojip_mean.T.groupby("Device").max().max(axis=1)

# Plot
fig, axes = plt.subplots(
    9,
    7,
    sharex=True, 
    # sharey=True,
    figsize=(20,20),
    width_ratios=[1,1,1,0.5,1,1,1]
)

cmap = cm.cool
colornorm = plt.Normalize(
light_intensities.min().min(),
light_intensities.max().max()
)

strain_map = {
    "syn": r"$\mathit{\boldsymbol{Synechocystis}}$",
    "chlo": r"$\mathit{\boldsymbol{Chlorella}}$"
}

device_map ={
    "MCPAM": "Multi\u2010Color\\ PAM",
    "FL6000": "FL\\ 6000"
}

plot_strains = ["syn", "chlo"]

for s, strain in enumerate(plot_strains):
    for d,device in enumerate(devices):
        ojip_points = ojip_points_res[device]["points"]
    
        # Create the colormap for the light intensity
        _ods = ods.loc[idx[strain, device, :], ["OD680", "Chlorella Chl a (mg/L)", "Synechocystis (mg/L)"]].sort_index(ascending=False)
        for o, (_, _row) in enumerate(_ods.iterrows()):

            od = _row["OD680"]

            if strain == "syn":
                chla = _row["Synechocystis (mg/L)"]
            else:
                chla = _row["Chlorella Chl a (mg/L)"]

            ax = axes[o, (s*4+d)]

            density_text = f"{chla:.2f} mg L$^{{-1}}$ Chl $\\mathit{{a}}$"

            if o==0:
                ax.set_title(f"$\\mathbf{{{device_map.get(device,device)}}}$\n{density_text}")
            else:
                ax.set_title(density_text)

            try:
                dat = ojip_mean.loc[:, idx[device, strain,:,od,:]].dropna()
            except KeyError:
                ax.grid(which="minor")
                ax.grid(which="major")
                continue

            _light_intensities = dat.columns.get_level_values(-1)
            _light_intensities = light_intensities.loc[_light_intensities, (device, strain)].to_numpy().reshape(-1,1)

            for nam, row in dat.T.iterrows():
                ax.plot(row, c=cmap(colornorm(light_intensities.loc[nam[-1],(device, strain)])))

            ax.plot(
                ojip_points.loc[idx[strain,:,od,:], ("inflection", "FJ_time")],
                ojip_points.loc[idx[strain,:,od,:], ("inflection", "FJ_value")],
                marker="o", fillstyle='none', c='k',
                ls="",
                markersize=4
            )

            ax.grid(which="minor")
            ax.grid(which="major")

            ax.axvspan(FJ_time_min, FJ_time_max, color="grey", alpha=0.5)


for strain, x in zip(plot_strains, [0.29, 0.73]):
    fig.text(x, 0.91, strain_map[strain], size=20, weight='bold', ha="center", style="italic")

for ax in axes[-1]:
    ax.set_xscale("log")
    ax.set_xlabel("Time (ms)")

for ax in axes[:,[0, 4]].flatten():
    ax.set_ylabel("Double normalized\nfluorescence (r.u.)")

def log_tick_formatter(val, pos):
    return f"{val:g}"  # Uses general format, removes scientific notation

for ax in axes[-1,:].flatten():
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))

for i in range(9):
    for j in range(1,3):
        axes[i,j].sharey(axes[0,0])
        axes[i,j].tick_params(labelleft=False)
        axes[i,j+4].sharey(axes[0,4])
        axes[i,j+4].tick_params(labelleft=False)

for ax in axes[:, 3].flatten():
    ax.remove()

fig.subplots_adjust(right=0.9)

# Add a colorbar
cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])

fig.colorbar(
    cm.ScalarMappable(norm=colornorm, cmap=cmap),
            cax=cax, orientation='vertical', label='Saturation pulse intensity (µmol Photons m$^{-2}$ s^{-1}')

fig.savefig("figures/NormOJIP_per_OD.png")
plt.show(fig)
plt.close(fig)

del ojip_mean
del row
gc.collect()

## Plot of raw OJIP

In [ ]:
#### PLOTTING ####
# Raw OJIP plots

ojip_mean = ojip

ojip_max = ojip_mean.T.groupby("Device").max().max(axis=1)

# Values for shading
FJ_time_min = feature_finding_options["FJ_time_min"]  # (float(str(request.form.get('FJ_time_min')))) 
FJ_time_max = feature_finding_options["FJ_time_max"]   # (float(str(request.form.get('FJ_time_max')))) 

# Plot
fig, axes = plt.subplots(
    9,
    7,
    sharex=True, 
    sharey=True,
    figsize=(20,20),
    width_ratios=[1,1,1,0.5,1,1,1]
)

cmap = cm.cool
colornorm = plt.Normalize(
light_intensities.min().min(),
light_intensities.max().max()
)

strain_map = {
    "syn": r"$\mathit{\boldsymbol{Synechocystis}}$",
    "chlo": r"$\mathit{\boldsymbol{Chlorella}}$"
}

plot_strains = ["syn", "chlo"]

device_map ={
    "MCPAM": "Multi\u2010Color\\ PAM",
    "FL6000": "FL\\ 6000"
}


for s, strain in enumerate(plot_strains):
    for d,device in enumerate(devices):
        ojip_points = ojip_points_res[device]["points"]
    
        # Create the colormap for the light intensity

        _ods = ods.loc[idx[strain, device, :], ["OD680", "Chlorella Chl a (mg/L)", "Synechocystis (mg/L)"]].sort_index(ascending=False)
        for o, (_, _row) in enumerate(_ods.iterrows()):

            od = _row["OD680"]

            if strain == "syn":
                chla = _row["Synechocystis (mg/L)"]
            else:
                chla = _row["Chlorella Chl a (mg/L)"]

            ax = axes[o, (s*4+d)]

            density_text = f"{chla:.2f} mg L$^{{-1}}$ Chl $\\mathit{{a}}$"

            if o==0:
                # ax.set_title(f"$\\boldsymbol{{{device}}}$\n{density_text}")
                ax.set_title(f"$\\mathbf{{{device_map.get(device,device)}}}$\n{density_text}")
            else:
                ax.set_title(density_text)

            try:
                dat = ojip_mean.loc[:, idx[device, strain,:,od,:]].dropna() /ojip_max[device]
            except KeyError:
                ax.grid(which="both")
                continue

            _light_intensities = dat.columns.get_level_values(-1)
            _light_intensities = light_intensities.loc[_light_intensities, (device, strain)].to_numpy().reshape(-1,1)

            for nam, row in dat.T.iterrows():
                ax.plot(row, c=cmap(colornorm(light_intensities.loc[nam[-1],(device, strain)])))
            
            # ax.axvline(1, c="k", ls="--")
            ax.grid(which="both")

            # ax.axvspan(FJ_time_min, FJ_time_max, color="grey", alpha=0.5)

for strain, x in zip(plot_strains, [0.29, 0.73]):
    fig.text(x, 0.91, strain_map[strain], size=20, weight='bold', ha="center")

for ax in axes[-1]:
    ax.set_xscale("log")
    ax.set_xlabel("Time (ms)")

for ax in axes[:,[0, 4]].flatten():
    ax.set_ylabel("Normalized\nfluorescence (r.u.)")

def log_tick_formatter(val, pos):
    return f"{val:g}"  # Uses general format, removes scientific notation

for ax in axes[-1,:].flatten():
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))

for ax in axes[:,4]:
    ax.tick_params(labelleft=True)

for ax in axes[:, 3].flatten():
    ax.remove()
        # axes[0,-1].legend(loc="upper left", bbox_to_anchor=(1,1), title="light intensity")

        # fig.tight_layout()
fig.subplots_adjust(right=0.9)

# Add a colorbar
cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])

# Include zero on y
ax.set_ylim(0)

fig.colorbar(
    cm.ScalarMappable(norm=colornorm, cmap=cmap),
            cax=cax, orientation='vertical', label='Saturation pulse intensity (µmol Photons m$^{-2}$ s^{-1}')

fig.savefig("figures/OJIP_per_OD.png")
plt.show(fig)
plt.close(fig)

del ojip_mean
del row
gc.collect()

## Plot of VJ

In [ ]:
#### PLOTTING ####
# Heatmap 
fig, axes = plt.subplots(3, len(strains), figsize=(7,10), sharex=True, sharey=True)

_VJ_min=np.repeat(np.nan, len(devices))
_VJ_max=np.repeat(np.nan, len(devices))
for d, device in enumerate(devices):
    dat = ojip_points_res[device]["points"].loc[:,("inflection", "FJ_value")]
    dat_times = ojip_points_res[device]["points"].loc[:,("inflection", "FJ_time")]
    dat_good = dat.loc[np.logical_and(
        dat_times > feature_finding_options["FJ_time_min"]*1.2,
        dat_times < feature_finding_options["FJ_time_max"]/1.2,
    )]

    _VJ_min[d]=dat_good.min()
    _VJ_max[d]=dat_good.max()

VJ_extrema=[_VJ_min.min(), _VJ_max.max()]

# Add 0.8 as upper max to the colorbar
VJ_extrema[1]=0.8

colornorm = plt.Normalize(*VJ_extrema)
cmap=cm.coolwarm
cmap.set_under("k")

plot_strains = ["syn", "chlo"]

plot_devices = ["FL6000", "MCPAM", "AquaPen"]

device_map ={
    "MCPAM": "Multi-Color PAM",
    "FL6000": "FL 6000"
}

for d, device in enumerate(plot_devices):

    for i,strain in enumerate(plot_strains):
        # Select the data to plot
        dat = ojip_points_res[device]["points"].copy()
        dat = dat.loc[idx[strain], ("inflection", "FJ_value")].droplevel(0)

        

        dat = dat.unstack(["OD680"])

        dat = dat.sort_index(axis=0)
        dat = dat.sort_index(axis=1)

        dat_times = ojip_points_res[device]["points"].copy()
        dat_times = dat_times.loc[idx[strain], ("inflection", "FJ_time")].droplevel(0)
        dat_times = dat_times.unstack(["OD680"])

        dat_times = dat_times.sort_index(axis=0)
        dat_times = dat_times.sort_index(axis=1)

        bad_time = np.logical_or(
            dat_times < feature_finding_options["FJ_time_min"]*1.2,
            dat_times > feature_finding_options["FJ_time_max"]/1.2,
        )

        # Replace values with bad timing assuming that no proper value could be determined
        dat[bad_time] = np.nan
        # dat[bad_time] = -100

        # Get the Light intensity and OD values
        X_name = "Light_intensity"
        x = dat.index.get_level_values("Light_intensity").to_numpy()
        X = light_intensities.loc[x, (device, strain)].to_numpy().reshape(-1,1)
        X = np.repeat(X, dat.shape[1], axis=1)

        # Y_name = "Chl a (mg L$^{-1}$)"
        Y_name = "Chl $\\mathit{a}$ (mg L$^{-1}$)"
        # Get Chla from OD680
        _ods = ods.loc[idx[strain, device, :]]
        Y= _ods.set_index("OD680").loc[
            dat.columns.get_level_values("OD680").to_numpy(), 
            "Synechocystis (mg/L)" if strain == "syn" else "Chlorella Chl a (mg/L)"
        ].to_numpy().reshape(1,-1)

        Y = np.repeat(Y, dat.shape[0], axis=0)

        Z = dat.to_numpy().astype(float)

        # Plot the surface
        ax = axes[d, i]

        surf = ax.pcolormesh(Y,X, colornorm(Z), cmap=cmap, vmin=0)

        ax.set_xscale("log")

        xlim = (
            ods.loc[:,["Chlorella Chl a (mg/L)", "Synechocystis (mg/L)"]].min().min(),
            ods.loc[:,["Chlorella Chl a (mg/L)", "Synechocystis (mg/L)"]].max().max()
        )
        ax.set_xlim(xlim * np.array([1/1.2, 1.2]))

        if d==0:
            ax.set_title(f"$\\boldsymbol{{{strain_map[strain]}}}$\n{device_map.get(device,device)}")
        else:
            ax.set_title(device_map.get(device,device))

        ax.set_box_aspect(1)
        ax.grid(which="minor", linewidth=0.4)
        ax.grid(which="major", linewidth=0.5)

for ax in axes[:,0]:
    # ax.set_ylabel("Light intensity\n[µmol Photons m$^{-2}$ s$^{-1}$]")
    ax.set_ylabel("Saturation pulse intensity\n(µmol photons m-2 s-1)")

# Set the x labels
def log_tick_formatter(val, pos):
    return f"{val:g}"  # Uses general format, removes scientific notation

for ax in axes[-1,:]:
    ax.set_xlabel(Y_name)
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))

for i, ax in enumerate(axes.flatten()):
    ax.text(0.05, 0.95,chr(i+65), transform=ax.transAxes, size=14, weight="bold", va="top", ha="left")

# Add a colorbar
cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(
    cm.ScalarMappable(norm=colornorm, cmap=cmap),
            cax=cax, orientation='vertical', label="V$_J$ (r.u.)")


fig.savefig("figures/VJ_OD_LightIntensity_heatmap.png", bbox_inches="tight")